In [ ]:
import os
os.chdir('../')

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

import pandas as pd
import pymc3 as pm

import clds

In [ ]:
def plot_stats(env_name, stats, plot_steps, logy=False):
    plt.rcParams.update({'font.size': 32})
    fig, ax = plt.subplots(figsize=(10, 8))
    if logy:
        ax.semilogy(stats[0,:plot_steps,-1]/N*100, label='median, {}'.format(env_name))
    else:
        ax.plot(stats[0,:plot_steps,-1]/N*100, label='median, {}'.format(env_name))
    ax.plot(stats[1,:plot_steps,-1]/N*100, label=' 75%, {}'.format(env_name))
    ax.plot(stats[2,:plot_steps,-1]/N*100, label=' 95%, {}'.format(env_name))
    ax.set_xlabel('time (days)')
    ax.set_ylabel('Total Infected People [%]')
    
    if logy:
        ylim = ax.get_ylim()
        ax.set_ylim(10e-3, ylim[1])
    ax.grid(True, axis='y')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.legend()
    plt.tight_layout()

# Sensitivity to Uncertainty in Model Parameters

## Sample Parameters

In [ ]:
# default latent parameters
alpha_ = 0.570
beta_ = 0.011
gamma_ = 0.456
delta_ = 0.011
abcd_ = alpha_ + beta_ + gamma_ + delta_

epsilon_ = 0.171
theta_ = 0.371
zeta_ = 0.125
eta_ = 0.125
mu_ = 0.012
nu_ = 0.027
tau_ = 0.003
h_ = 0.034
rho_ = 0.034
kappa_ = 0.017
xi_ = 0.017
sigma_ = 0.017

n_samples = 1000
default_sd = 0.1 # default relative standard deviation
outfile = 'results/sidarthe_sensitivity_r0_samples.npz'

with pm.Model() as model:    
    # Distribution parameters taken from meta analysis in [1]
    R0_target = pm.TruncatedNormal('R0', mu=2.675739, sd=0.5719293, lower=0)
    
    # Noisy samples around parameters taken from [4]
    alpha = pm.TruncatedNormal('alpha_', mu=alpha_, sd=alpha_*default_sd, lower=0)
    beta = pm.TruncatedNormal('beta_', mu=beta_, sd=beta_*default_sd, lower=0)
    gamma = pm.TruncatedNormal('gamma_', mu=gamma_, sd=gamma_*default_sd, lower=0)
    delta = pm.TruncatedNormal('delta_', mu=delta_, sd=delta_*default_sd, lower=0)
    epsilon = pm.TruncatedNormal('epsilon', mu=epsilon_, sd=epsilon_*default_sd, lower=0)
    theta = pm.TruncatedNormal('theta', mu=theta_, sd=theta_*default_sd, lower=0)
    zeta = pm.TruncatedNormal('zeta', mu=zeta_, sd=zeta_*default_sd, lower=0)
    eta = pm.TruncatedNormal('eta', mu=eta_, sd=eta_*default_sd, lower=0)
    mu = pm.TruncatedNormal('mu', mu=mu_, sd=mu_*default_sd, lower=0)
    nu = pm.TruncatedNormal('nu', mu=nu_, sd=nu_*default_sd, lower=0)
    tau = pm.TruncatedNormal('tau', mu=tau_, sd=tau_*default_sd, lower=0)
    h = pm.TruncatedNormal('h', mu=h_, sd=h_*default_sd, lower=0)
    rho = pm.TruncatedNormal('rho', mu=rho_, sd=rho_*default_sd, lower=0)
    kappa = pm.TruncatedNormal('kappa', mu=kappa_, sd=kappa_*default_sd, lower=0)
    xi = pm.TruncatedNormal('xi', mu=xi_, sd=xi_*default_sd, lower=0)
    sigma = pm.TruncatedNormal('sigma', mu=sigma_, sd=sigma_*default_sd, lower=0)
    
    # rescale R0 to target
    r1 = epsilon + zeta + h
    r2 = eta + rho
    r3 = theta + mu + kappa
    r4 = nu + xi
    r5 = sigma + tau
    r0 = alpha / r1
    r0 += beta * epsilon / (r1 * r2)
    r0 += gamma * zeta / (r1 * r3)
    r0 += delta * eta * epsilon / (r1 * r2 * r4)
    r0 += delta * zeta * theta / (r1 * r3 * r4)
    
    alpha = pm.Deterministic('alpha', alpha/r0 * R0_target)
    beta = pm.Deterministic('beta', beta/r0 * R0_target)
    gamma = pm.Deterministic('gamma', gamma/r0 * R0_target)
    delta = pm.Deterministic('delta', delta/r0 * R0_target)
    
    prior = pm.sample_prior_predictive(n_samples)
    
np.savez(outfile, **prior)

## Determine switching start from lockdown policy without switching

For a fair comparison, we continue prolonged lockdown beyond day 50 until the number of observably infected people reduces to that observed with the median parameter on day 50. In order to determine the sample-specific start of switching, we run a first simulation with prolonged lockdown and no switching at all, and save the switching start date for future use.

In [ ]:
source_file = 'results/sidarthe_sensitivity_r0_samples.npz'
# load samples
samples = {}
with np.load(source_file) as f:
    for x in f:
        samples[x] = f[x]
        
# ODE
batch_size = samples['R0'].shape[0]
alpha = samples['alpha']
beta = samples['beta']
gamma = samples['gamma']
delta = samples['delta']

# initial condition
N=1e7
I = 500/6
D = 20
A = 1
R = 2
T = H = E = 0
S = N - I - D - A - R - T - H - E
s0 = np.array([S, I, D, A, R, T, H, E])
step_size=0.001

lockdown_effectiveness = 0.175
suppression_start = 20
suppression_end = None
beta_high = 1.

model = clds.agents.BatchSIDARTHE(s0=s0, 
                    alpha='a',
                    beta='b',
                    gamma='g',
                    delta='d',
                    epsilon=samples['epsilon'],
                    theta=samples['theta'],
                    zeta=samples['zeta'],
                    eta=samples['eta'],
                    mu=samples['mu'],
                    nu=samples['nu'],
                    tau=samples['tau'],
                    h=samples['h'],
                    rho=samples['rho'],
                    kappa=samples['kappa'],
                    xi=samples['xi'],
                    sigma=samples['sigma'],
                    N=N, 
                    batch_size=batch_size,
                    step_size=step_size)

policy = clds.agents.BatchLockdown(beta_high=beta_high,
                 beta_low=lockdown_effectiveness*beta_high, 
                 suppression_start=suppression_start,
                 suppression_end=suppression_end,
                 batch_size=1)

env = clds.Composite()
env.add(model, 
        pre=lambda x: {'a': x['policy']*alpha,
                       'b': x['policy']*beta,
                       'g': x['policy']*gamma,
                       'd': x['policy']*delta},
        out='model')
env.add(policy, out='policy')

labels = ['S', 'I', 'D', 'A', 'R', 'T', 'H', 'E']
n_steps = 350
env_out = [env.reset()] + [env.step()[0] for _ in range(n_steps)]
s_policy = np.array([o['policy'] for o in env_out]).swapaxes(0,1)
env_out = np.array([o['model'] for o in env_out]).swapaxes(0,1)
infected = np.sum(env_out[:,::,1:6], axis=2) # sum over I D A R T
detected = np.sum(env_out[:,::,[2, 4, 5]], axis=2) # sum over D R T

env_out = np.concatenate([env_out, infected.reshape(infected.shape + (1,))], axis=2)
env_out = np.concatenate([env_out, detected.reshape(detected.shape + (1,))], axis=2)
env_stats = np.quantile(env_out, q=[0.5, 0.75, 0.95], axis=0)

filename = 'results/sidarthe_sensitivity_ldp_step_{}'.format(step_size)

switching_start=50
# select switching start by number of quarantined
median_detected = env_stats[0,switching_start,-1]
switching_start = np.argmax(env_out[:,switching_start:,-1] <=median_detected, axis=1)+switching_start
print('detected(50) median', median_detected, median_detected/N*100)
np.savez(filename+'.npz', states=env_out, stats=env_stats, betas=s_policy, switching_start=switching_start)

plot_stats('LDP', env_stats, n_steps, logy=True)
ylim = plt.ylim()
plt.plot([suppression_start, suppression_start], ylim, '--', c='red')
plt.subplots(figsize=(10, 8))
plt.hist(switching_start, bins=10, density=True)
plt.xlabel('lockdown end (day)');

## Open Loop

In [ ]:
source_file = 'results/sidarthe_sensitivity_r0_samples.npz'
# load samples
samples = {}
with np.load(source_file) as f:
    for x in f:
        samples[x] = f[x]
        
# FPSP
suppression_start = 20
filename = 'results/sidarthe_sensitivity_ldp_step_0.001.npz'
with np.load(filename) as f:
    switching_start = f['switching_start']
        
# ODE
batch_size = samples['R0'].shape[0]
alpha = samples['alpha']
beta = samples['beta']
gamma = samples['gamma']
delta = samples['delta']

# initial condition
N=1e7
I = 500/6
D = 20
A = 1
R = 2
T = H = E = 0
S = N - I - D - A - R - T - H - E
s0 = np.array([S, I, D, A, R, T, H, E])
step_size=0.001

lockdown_effectiveness = 0.175
suppression_start = 20
n_steps = 1000

policies = [(4, 10), (5,9), (6, 8),
            (8, 20), (10, 18), (12, 16),
            (16, 40), (20, 36), (24, 32),
            (32, 80), (40, 72), (48, 64)]

for steps_high, steps_low in policies:
    if (steps_low == 0) and (steps_high == 0):
        continue
    if (steps_low == 0) and (steps_high > 1):
        continue
    if (steps_high == 0) and (steps_low > 1):
        continue

    print('FPSP-(', steps_high, ',', steps_low, ')')  

    model = clds.agents.BatchSIDARTHE(s0=s0, 
                        alpha='a',
                        beta='b',
                        gamma='g',
                        delta='d',
                        epsilon=samples['epsilon'],
                        theta=samples['theta'],
                        zeta=samples['zeta'],
                        eta=samples['eta'],
                        mu=samples['mu'],
                        nu=samples['nu'],
                        tau=samples['tau'],
                        h=samples['h'],
                        rho=samples['rho'],
                        kappa=samples['kappa'],
                        xi=samples['xi'],
                        sigma=samples['sigma'],
                        N=N, 
                        batch_size=batch_size,
                        step_size=step_size)

    policy = clds.agents.BatchFPSP(beta_high=1,
                         beta_low=lockdown_effectiveness, 
                         steps_high=steps_high,
                         steps_low=steps_low, 
                         suppression_start=suppression_start, 
                         switching_start=switching_start, 
                         batch_size=batch_size)

    env = clds.Composite()
    env.add(model, 
            pre=lambda x: {'a': x['policy']*alpha,
                           'b': x['policy']*beta,
                           'g': x['policy']*gamma,
                           'd': x['policy']*delta},
            out='model')
    env.add(policy, out='policy')

    labels = ['S', 'I', 'D', 'A', 'R', 'T', 'H', 'E']
    
    env_out = [env.reset()] + [env.step()[0] for _ in range(n_steps)]
    s_policy = np.array([o['policy'] for o in env_out]).swapaxes(0,1)
    env_out = np.array([o['model'] for o in env_out]).swapaxes(0,1)
    infected = np.sum(env_out[:,::,1:6], axis=2) # sum over I D A R T
    env_out = np.concatenate([env_out, infected.reshape(infected.shape + (1,))], axis=2)
    env_stats = np.quantile(env_out, q=[0.5, 0.75, 0.95], axis=0)

    filename = 'results/sidarthe_fpsp_{}_var_{}_{}_{}_sensitivity_batch_{}_step{}x{}'.format(suppression_start, 
                                                                         steps_high, 
                                                                         steps_low, 
                                                                         lockdown_effectiveness,
                                                                         batch_size, 
                                                                         n_steps,
                                                                         step_size)
    
    env_name = 'FPSP-({}, {})'.format(steps_high, steps_low)
    plot_stats(env_name, env_stats, n_steps)
    ylim = plt.ylim()
    plt.plot([suppression_start, suppression_start], ylim, '--', c='red')
    plt.plot([50, 50], ylim, '--', c='blue')
    plt.savefig(filename+'.eps', dpi=1200)
    plt.savefig(filename+'.png', dpi=300)

## Supervisory Outer Loop

In [ ]:
source_file = 'results/sidarthe_sensitivity_r0_samples.npz'
# load samples
samples = {}
with np.load(source_file) as f:
    for x in f:
        samples[x] = f[x]

# outer supervisory control
alpha_x = 0.4 # hystheresis for increasing duty cycle
alpha_y = 0. # hystheresis for decreasing duty cycle
x_init = 0
x_max = 14
period = x_max        
        
# FPSP
suppression_start=20
steps_high = 1
steps_low = 6
lockdown_effectiveness=0.175

filename = 'results/sidarthe_ldp_step_0.001.npz'
with np.load(filename) as f:
    switching_start = f['switching_start']
        
# ODE
batch_size = samples['R0'].shape[0]
alpha = samples['alpha']
beta = samples['beta']
gamma = samples['gamma']
delta = samples['delta']

# initial condition
N=1e7
I = 500/6
D = 20
A = 1
R = 2
T = H = E = 0
S = N - I - D - A - R - T - H - E
s0 = np.array([S, I, D, A, R, T, H, E])
step_size=0.001

lockdown_effectiveness = 0.175
suppression_start = 20
n_steps = 1000

policies = [#7 
            14,
            28,
            58,
            112]

for x_max in policies:
    print('T', x_max) 
    
    period = x_max
    model = clds.agents.BatchSIDARTHE(s0=s0, 
                        alpha='a',
                        beta='b',
                        gamma='g',
                        delta='d',
                        epsilon=samples['epsilon'],
                        theta=samples['theta'],
                        zeta=samples['zeta'],
                        eta=samples['eta'],
                        mu=samples['mu'],
                        nu=samples['nu'],
                        tau=samples['tau'],
                        h=samples['h'],
                        rho=samples['rho'],
                        kappa=samples['kappa'],
                        xi=samples['xi'],
                        sigma=samples['sigma'],
                        N=N, 
                        batch_size=batch_size,
                        step_size=step_size)
    # outer loop feedback: D+R
    u = clds.Lambda(reset_fn= lambda: 0, step_fn= lambda x: x['model'][:,2] + x['model'][:,4]) # batch, channel
    
    outer_loop = clds.agents.BatchOuterLoopFPSP(start=switching_start,
                            o='o', 
                            period=period, 
                            x_init=x_init,
                            x_max=x_max, 
                            alpha_x=alpha_x, 
                            alpha_y=alpha_y, 
                           batch_size=batch_size)
    
    policy = clds.agents.BatchFPSP(beta_high=1,
                         beta_low=lockdown_effectiveness, 
                         steps_high=steps_high,
                         steps_low=steps_low, 
                         suppression_start=suppression_start, 
                         switching_start=switching_start, 
                         batch_size=batch_size)

    env = clds.Composite(order='sequential')
    env.add(model, 
            pre=lambda x: {'a': x['policy']*alpha,
                           'b': x['policy']*beta,
                           'g': x['policy']*gamma,
                           'd': x['policy']*delta},
            out='model')
    env.add(u, out='o')
    env.add(outer_loop, out='ol')
    env.add(policy, out='policy', pre= lambda x: {'x': x['ol'][:,0], 'y': x['ol'][:,1]})

    labels = ['S', 'I', 'D', 'A', 'R', 'T', 'H', 'E']
    
    env_out = [env.reset()] + [env.step()[0] for _ in range(n_steps)]
    s_policy = np.array([o['policy'] for o in env_out]).swapaxes(0,1)
    s_ol = np.array([o['ol'] for o in env_out]).swapaxes(0,1)
    env_out = np.array([o['model'] for o in env_out]).swapaxes(0,1)
    infected = np.sum(env_out[:,::,1:6], axis=2) # sum over I D A R T
    env_out = np.concatenate([env_out, infected.reshape(infected.shape + (1,))], axis=2)
    env_stats = np.quantile(env_out, q=[0.5, 0.75, 0.95], axis=0)

    filename = 'results/sidarthe_fpsp_{}_var_T_{}_{}_sensitivity_batch_{}_step{}x{}'.format(suppression_start, 
                                                                         x_max,
                                                                         lockdown_effectiveness,
                                                                         batch_size, 
                                                                         n_steps,
                                                                         step_size)

    env_name = 'T={}'.format(x_max)
    plot_stats(env_name, env_stats, n_steps)
    ylim = plt.ylim()
    plt.plot([suppression_start, suppression_start], ylim, '--', c='red')
    plt.plot([50, 50], ylim, '--', c='blue')
    plt.savefig(filename+'.eps', dpi=1200)
    plt.savefig(filename+'.png', dpi=300)